In [9]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Data preparation
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 10% of the data for validation
)

train_generator = train_datagen.flow_from_directory(
    'dataset',  # Update this to your dataset directory
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',  # Update this to your dataset directory
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 7246 images belonging to 2 classes.
Found 1811 images belonging to 2 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # Convolutional layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Convolutional layer 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Convolutional layer 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Fully connected classifier
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')  # Assuming 2 classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_6 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_7 (Bat  (None, 109, 109, 64)      256       
 chNormalization)                                                
                                                      

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Training the model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model_custom.keras', save_best_only=True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

Epoch 1/10
227/227 [==============================] - 325s 1s/step - loss: 2.1434 - accuracy: 0.7381 - val_loss: 5.0029 - val_accuracy: 0.2358 - lr: 0.0010
Epoch 2/10
227/227 [==============================] - 354s 2s/step - loss: 0.5275 - accuracy: 0.7648 - val_loss: 0.8423 - val_accuracy: 0.7493 - lr: 0.0010
Epoch 3/10
227/227 [==============================] - 384s 2s/step - loss: 0.5921 - accuracy: 0.7599 - val_loss: 0.6275 - val_accuracy: 0.6753 - lr: 0.0010
Epoch 4/10
227/227 [==============================] - 389s 2s/step - loss: 0.4965 - accuracy: 0.7624 - val_loss: 0.6360 - val_accuracy: 0.7637 - lr: 0.0010
Epoch 5/10
227/227 [==============================] - 402s 2s/step - loss: 0.4784 - accuracy: 0.7641 - val_loss: 0.5423 - val_accuracy: 0.7686 - lr: 0.0010
Epoch 6/10
227/227 [==============================] - 409s 2s/step - loss: 0.5079 - accuracy: 0.7635 - val_loss: 0.5399 - val_accuracy: 0.7548 - lr: 0.0010
Epoch 7/10
227/227 [==============================] - 401s 2s/st

In [13]:
from tensorflow.keras.models import load_model

# Assuming your model is saved under 'best_model.keras'
model = load_model('best_model_custom.keras')


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32  # Adjust to your preference
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory='test_dataset_processed',
    target_size=(224, 224),  # Ensure this matches your model's expected input
    batch_size=batch_size,
    class_mode='categorical',  # or 'binary' for binary classification
    shuffle=False  # Important for confusion matrix
)


Found 1090 images belonging to 2 classes.


In [15]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

35/35 [==============================] - 11s 273ms/step - loss: 0.5033 - accuracy: 0.7275
Test Loss: 0.5032638311386108
Test Accuracy: 0.7275229096412659
